# MLlib: Basic Statistics and Exploratory Data Analysis

We will introduce Spark's machine learning library [MLlib](https://spark.apache.org/docs/latest/mllib-guide.html).

## Getting the data and creating the RDD

The main objective is to play with a small dataset from the Hackathon.

# Prepare the data

## Currently, do not run any command. 

Re-run it at the end if you have the time. For now, let's just see how to create a dataframe.

First, install the h5py if you don't have it.

In [4]:
!sudo pip install h5py

    100% |████████████████████████████████| 4.8MB 258kB/s eta 0:00:01


In [1]:
import h5py
f = h5py.File("eightieth.h5", "r")

The content of the file:

In [2]:
X, Y = f["S2"], f["TOP_LANDCOVER"]

Extract images from np arrays

In [ ]:
ims = list()
lbls = list()

for i in range(X.shape[0]):
    im=X[i,::]
    lbl=Y[i]
    
    ims.append(im)
    lbls.append(lbl)

In [97]:
data = sc.parallelize(zip(ims, lbls))

Compute some simple features from an image. Returns list of features and a label.

In [25]:
from pyspark import Row

def compute_features(t):
    im, lbl = t
    
    means = im.mean(axis=(0, 1)).astype(float).tolist()
    
    stds = im.std(axis=(0, 1)).astype(float).tolist()
    
    return means+stds+lbl.astype(int).tolist()

In [26]:
compute_features((im, lbl))

[620.60546875,
 859.84375,
 928.0703125,
 2435.7265625,
 245.32203674316406,
 122.14816284179688,
 110.50434875488281,
 184.1570281982422,
 5]

Transform it to a dataframe. Requires giving a name to columns.

In [ ]:
mean_names = ["Mean_{}".format(i) for i in range(4)]
    
std_names = ["Std_{}".format(i) for i in range(4)]

df = data.map(compute_features).toDF(mean_names+std_names+["Label"])

Writing to the Google Storage (HDFS).

In [ ]:
df.write.parquet("gs://mlg-store/isae/im-features-all.parquet")

In [ ]:
df.count()

# The hands on starts here

Now you can execute commands.

# Processing
A dataset is already available to use: 

Links:
* https://storage.googleapis.com/mlg-store/isae/im-features-100000.parquet/_SUCCESS
* https://storage.googleapis.com/mlg-store/isae/im-features-100000.parquet/part-00000-6685d2e1-601f-490f-be13-586b1b0b260d-c000.snappy.parquet
* https://storage.googleapis.com/mlg-store/isae/im-features-100000.parquet/part-00001-6685d2e1-601f-490f-be13-586b1b0b260d-c000.snappy.parquet

Please respect the order, Root folder:
* ./im-features-100000.parquet/
    * _SUCCESS
    * part-00000-6685d2e1-601f-490f-be13-586b1b0b260d-c000.snappy.parquet
    * part-00001-6685d2e1-601f-490f-be13-586b1b0b260d-c000.snappy.parquet
                                

In [94]:
featuresDF=None
featuresDF = sqlContext.read.parquet("./im-features-100000.parquet")

First, parsing the file...

In [95]:
featuresDF.printSchema()

root
 |-- Mean_0: double (nullable = true)
 |-- Mean_1: double (nullable = true)
 |-- Mean_2: double (nullable = true)
 |-- Mean_3: double (nullable = true)
 |-- Std_0: double (nullable = true)
 |-- Std_1: double (nullable = true)
 |-- Std_2: double (nullable = true)
 |-- Std_3: double (nullable = true)
 |-- Label: long (nullable = true)



In [33]:
result = featuresDF.describe().collect()

In [34]:
for l in result:
    print "----------------------"
    r = l.asDict()
    print "Statistics {}".format(r["summary"])
    for key in r.keys():
        print "{0}: {1}".format(key, r[key])
    print "----------------------"

----------------------
Statistics count
summary: count
Std_3: 100
Std_2: 100
Std_1: 100
Std_0: 100
Label: 100
Mean_0: 100
Mean_1: 100
Mean_2: 100
Mean_3: 100
----------------------
----------------------
Statistics mean
summary: mean
Std_3: 184.23522521972657
Std_2: 77.61876323699951
Std_1: 92.73744741439819
Std_0: 167.9272707939148
Label: 6.26
Mean_0: 853.62453125
Mean_1: 994.3158203125
Mean_2: 1030.4110546875
Mean_3: 2450.00140625
----------------------
----------------------
Statistics stddev
summary: stddev
Std_3: 95.2802802807138
Std_2: 42.73152720352641
Std_1: 51.76375618880337
Std_0: 94.95745800620809
Label: 3.737930070523151
Mean_0: 244.62828105437603
Mean_1: 137.94634200835668
Mean_2: 104.88589335373015
Mean_3: 221.83929702320293
----------------------
----------------------
Statistics min
summary: min
Std_3: 61.98518371582031
Std_2: 17.851411819458008
Std_1: 25.63669776916504
Std_0: 30.777761459350586
Label: 2
Mean_0: 492.98046875
Mean_1: 788.02734375
Mean_2: 871.04296875
Mea

Compute statistics by labels

In [36]:
featuresDF.select("label").groupBy("label").count().show()

+-----+-----+
|label|count|
+-----+-----+
|    5|   41|
|    3|   17|
|   12|   28|
|    2|   11|
|    4|    3|
+-----+-----+



## Machine learning with Apache Spark
Now that the inputs are defined, we can apply some basics (or advanced) data processing functions to classify the type of interactions (i.e. "label")

In [68]:
from pyspark.ml.feature import StringIndexer
col_names=featuresDF.columns
col_names.remove("Label")
s = StringIndexer(inputCol="Label", outputCol="idx_label").fit(featuresDF)

In [45]:
result = s.transform(featuresDF)

In [47]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import VectorAssembler, PCA

assemblor = VectorAssembler(inputCols=col_names, outputCol="features")
rf = RandomForestClassifier(featuresCol="features", labelCol="idx_label", maxDepth=1, maxBins=32, numTrees=1)
pipeline = Pipeline(stages=[s, assemblor, rf])

Train and test splits

In [48]:
train, test = featuresDF.randomSplit([0.6,0.4])

In [49]:
model = pipeline.fit(train)

Compute accuracy on both train and test sets

In [50]:
model.transform(test).select("prediction", "idx_label").groupBy("prediction", "idx_label").count().show()

+----------+---------+-----+
|prediction|idx_label|count|
+----------+---------+-----+
|       1.0|      1.0|   10|
|       0.0|      0.0|   19|
|       0.0|      2.0|    7|
|       0.0|      3.0|    3|
+----------+---------+-----+



In [51]:
preds = model.transform(test)
print preds.where(preds.prediction == preds.idx_label).count()

29


## Exercice:
* Compute classic multi classification statistics precision recall etc ... for each label using spark 

Hint: compute the confusion matrix in spark and the final divisions in local

## Preprocessing

Try applying a PCA before learning the model

In [52]:
from pyspark.ml.feature import PCA

pca = PCA(k=2, inputCol="features", outputCol="pca_features")
assemblor = VectorAssembler(inputCols=col_names, outputCol="features")
rf = RandomForestClassifier(featuresCol="pca_features", labelCol="idx_label", maxDepth=1, maxBins=32, numTrees=1)
pipeline = Pipeline(stages=[s, assemblor, pca, rf])

In [53]:
model = pipeline.fit(train)

In [54]:
model.transform(test).select("prediction", "idx_label").groupBy("prediction", "idx_label").count().show()

+----------+---------+-----+
|prediction|idx_label|count|
+----------+---------+-----+
|       1.0|      1.0|    7|
|       0.0|      1.0|    3|
|       1.0|      0.0|   14|
|       1.0|      2.0|    4|
|       0.0|      0.0|    5|
|       1.0|      3.0|    3|
|       0.0|      2.0|    3|
+----------+---------+-----+



Try applying a kmeans to the dataset

In [55]:
from pyspark.ml.clustering import KMeans
kmeans = KMeans(k=2, seed=1, featuresCol="features", predictionCol="kmeans_pred")
assemblor = VectorAssembler(inputCols=col_names, outputCol="features")
kmeans_assemblor = VectorAssembler(inputCols=col_names+["kmeans_pred"], outputCol="kmeans_features")
rf = RandomForestClassifier(featuresCol="kmeans_features", labelCol="idx_label", maxDepth=1, maxBins=32, numTrees=1)
pipeline = Pipeline(stages=[s, assemblor, kmeans, kmeans_assemblor, rf])

In [56]:
model = pipeline.fit(train)

### Cross validation

In [73]:
from pyspark.ml.feature import PCA
s = StringIndexer(inputCol="Label", outputCol="idx_label")
pca = PCA(k=2, inputCol="features", outputCol="pca_features")
assemblor = VectorAssembler(inputCols=col_names, outputCol="features")
rf = RandomForestClassifier(featuresCol="pca_features", labelCol="idx_label", maxDepth=1, maxBins=32, numTrees=1)
pipeline = Pipeline(stages=[s, assemblor, pca, rf])


In [76]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

paramGrid = ParamGridBuilder() \
    .addGrid(pca.k, [2, 5, 8]) \
    .addGrid(rf.maxDepth, [1, 10]) \
    .build()

crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(labelCol="Label"),
                          numFolds=3)  # use 3+ folds in practice

# Run cross-validation, and choose the best set of parameters.
cvModel = crossval.fit(train)

In [77]:
cvModel.transform(test).select("prediction", "idx_label").groupBy("prediction", "idx_label").count().show()

+----------+---------+-----+
|prediction|idx_label|count|
+----------+---------+-----+
|       1.0|      1.0|    7|
|       0.0|      1.0|    3|
|       1.0|      0.0|   14|
|       1.0|      2.0|    4|
|       0.0|      0.0|    5|
|       1.0|      3.0|    3|
|       0.0|      2.0|    3|
+----------+---------+-----+



## Exercices:
### Detect classification noise using multiple models
    - If different models keep giving wrong predictions on the same sample, it may be a labeling mistake
    - Estimate labeling mistakes

### Implement the Viola Jones strategy using Spark
    - Learn a classifier (Decision Tree)
    - Reduce to zero the FPR (False alarms) using a threshold on probability
    - Keep only the examples where the classifier gives a positive answer
    - Repeat n times
    
### Hackathon training:
    - Change the features computed on images to increase performance (see first part of the notebook)
        - Only means and stds have been used here
    - Be sure to grab the raw dataset before hand [here](https://storage.googleapis.com/mlg-store/isae/eightieth.h5)
    